### ![Alt](http://images.squarespace-cdn.com/content/v1/5492d7f4e4b00040889988bd/1419973085209-9127JQ4DLEDQLNVH4FKT/PyomoNewBlueDense.png?format=300w "Pyomo")

# Pyomo Software for ECE1773 Power Systsem Operation and Control  and  ECE2781– Smart Grid Technology and Applications 

## Masoud Barati
March 01, 2022

github repository for workshop materials https://github.com/msdbarati/Power-system-optimization-and-its-applications

## How to solve a mathematical programming problem ?
1. AML(Algebraic Modeling Language)
    - GAMS(General Algebraic Modeling System)
        - first AML- 1976
        - widely accepted and used
        - old syntax
        - commercial
    - AMPL(A Mathematical Programming Language)
        - 1985 by Bob Fourer
        - higher level syntax
        - commercial
    - OPL(Optimization Programming Language)
        - uses javascript as scripting language
        - limited to cplex solver
        - commercial
2. Solver API
     - perfect compatibility
     - learning effort for each API
     - sometimes low-level
3. Modeling Libraries (like Pyomo)
     - not limited to a specific solver
     - more modeling features compared to API
     - integration with other algorithms and tools
     - data manipulation and visualization
     - no need to learn a new language

## What is Pyomo?
 - An open source python library for modeling mathematical programming problems
 - Linked to a wide range of commercial and free solvers
 - Developed as part of the COIN-OR(Computational Infrastructure of Operations Research) project

## Installing pyomo
it is **recommended** to use virtual environments
```console
> python -m venv env
> env\Scripts\activate
```
install pyomo package (from pypi)
```console
> pip install pyomo
```
check if pyomo is properly installed
```console
> pyomo --version
```
the output should be something like this
```console
> Pyomo 5.7 (CPython 3.8.2 on Windows 10)
```

## Installing  Solvers

- GLPK(GNU Linear Programming Kit)
    - LP/MIP
    - part of the GNU Project
    - developed by Andrew O. Makhorin of the Moscow Aviation Institute
    - download GLPK binaries for Windows from [glpk]
    - extract it in an arbitrary address (glpk_path)
    - the executable file path is {glpk_path}\w64\glpsol.exe
    - stable but slow
- CBC (COIN-OR Branch and Cut)
    - LP/MIP
    - part of the COIN-OR project
    - download CBC binaries for Windows from [cbc]
    - extract it in an arbitrary path (cbc_path)
    - the executable file path is {cbc_path}\bin\cbc.exe
- IPOPT (Interior Point Optimizer)
    - Nonlinear programming
    - part of the COIN-OR project
    - download CBC binaries for Windows from [ipopt]
    - extract it in an arbitrary path (ipopt_path)
    - the executable file path is {ipopt_path}\bin\ipopt.exe
- CPLEX (Commercial solver)
    - LP/MIP/QP/QCP
    - free Academic license
    - download and install from [cplex]
    - the executable file path is something like this
        - C:\Program Files\IBM\ILOG\CPLEX_Enterprise_Server1210\CPLEX_Studio\cplex\bin\x64_win64\cplex.exe
    

[glpk]: https://sourceforge.net/projects/winglpk/files/latest/download.
[cbc]: https://github.com/coin-or/Cbc/releases
[ipopt]: https://github.com/coin-or/Ipopt/releases
[cplex]: https://p30download.ir/fa/entry/62101/

## First Model
$$
\begin{align}
min \quad &x +y\\
s.t.\quad &x +4y <= 1\\
\quad &4x +y<= 1\\
\quad &x;y ≥ 0
\end{align}
$$

you can see the visualization [here]

[here]: https://www.desmos.com/calculator/a70y4wlpxy

In [38]:
## from pyomo.env import * (don't import like this, you may see this method of import in forums)
import pyomo.environ as pyo

## constructing the model object
model = pyo.ConcreteModel()

## Variables
model.x = pyo.Var(within=pyo.NonNegativeReals)
model.y = pyo.Var(domain=pyo.NonNegativeReals)
model.x.domain = pyo.Integers
## Constraints
model.con1 = pyo.Constraint(expr=model.x + 4*model.y <= 1)
model.con2 = pyo.Constraint(expr=4*model.x + model.y <= 1)

## Objective
# model.obj = pyo.Objective(expr=model.x + model.y)
model.obj = pyo.Objective(expr=model.x + model.y, sense=pyo.maximize)

##debugging the model
# model.pprint()
# model.x.pprint()
model.obj.pprint()
# model.con1.pprint()

# solvername = 'glpk'
# solverpath_exe = 'D:\\Solver\\winglpk-4.65\\glpk-4.65\\w64\\glpsol' # .exe

solvername='cplex'
solverpath_exe='C:\\Program Files\\IBM\\ILOG\\CPLEX_Enterprise_Server1210\\CPLEX_Studio\\cplex\\bin\\x64_win64\\cplex'

## Connecting to Solver and solving the instance
solver= pyo.SolverFactory(solvername, executable=solverpath_exe)


results = solver.solve(model)
# print(results)
## retrieving optimal values
print(pyo.value(model.obj))
print(pyo.value(model.x))
print(f"x: {pyo.value(model.x)}, y: {pyo.value(model.y)}")



obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : x + y
0.25
0.0
x: 0.0, y: 0.25


## Relaxing and Adding Constraints

$$
\begin{align}
min \quad &x +y\\
s.t.\quad &x +4y <= 1\\
\quad &5x +4y<= 3\\
\quad &x;y ≥ 0
\end{align}
$$


In [16]:
## deactivate constarints
model.con2.deactivate()
solver.solve(model)
print("After deactivating con2:")
print(f"x: {pyo.value(model.x)}, y: {pyo.value(model.y)}")

model.con3 = pyo.Constraint(expr=5*model.x + 4*model.y <= 3)
solver.solve(model)
print("After adding con3:")
print(f"x: {pyo.value(model.x)}, y: {pyo.value(model.y)}")

AttributeError: 'ConcreteModel' object has no attribute 'con2'

## Other Solvers

In [ ]:
# solvername = 'cbc'
# solverpath_exe = 'D:\\Solver\\Cbc-releases.2.10.7-w64-msvc15-md\\bin\\cbc'

# solvername='cplex'
# solverpath_exe='C:\\Program Files\\IBM\\ILOG\\CPLEX_Enterprise_Server1210\\CPLEX_Studio\\cplex\\bin\\x64_win64\\cplex'

# solvername = 'ipopt'
# solverpath_exe = 'D:\\Solver\\Ipopt-3.14.3-win64-msvs2019-md\\bin\\ipopt'

## Indexed Components
warehouse location problem
$$
\begin{align}
min \quad &\sum_{n\in N}\sum_{m \in M}d_{n,m} x_{n,m}\\
s.t.\quad &\sum_{n \in N} x_{n,m} =1 ,\quad \forall m \in M &(1)\\
\quad &  x_{n,m} \leq y_n, \quad \forall n \in N, \forall m \in M &(2)\\
\quad & \sum_{n \in N} y_n \leq P &(3)\\
\quad & 0\leq x_{n,m} \leq 1, \quad \forall n \in N, \forall m \in M\\
\quad & y_{n} \in \{0,1\}, \quad \forall n \in N\\
\end{align}
$$
<br>
<br>
![Alt](./images/WLP.png "wlp")

In [24]:

N = ["Harlingen", "Memphis", "Ashland"]
M = ["NYC", "LA", "CHI", "HTX"]
P = 2
d = {
    'Harlingen': {
        'NYC': 1956,
        'LA' : 1606,
        'CHI': 1410,
        'HTX': 330,
    },
    'Memphis': {
        'NYC': 1096,
        'LA' : 1792,
        'CHI': 531,
        'HTX': 567,
    },
    'Ashland': {
        'NYC': 485,
        'LA': 2322,
        'CHI': 324,
        'HTX': 1236,
    },
}

    
model = pyo.ConcreteModel()

## Variables
model.x = pyo.Var(N, M, bounds=(0,1), within=pyo.Reals, initialize = 1)
## alternative method for defining the x variables
# model.x = pyo.Var(N, M, within=pyo.PercentFraction)


initialize_dict = {
    "Harlingen": 1,
    "Memphis": 0,
    "Ashland": 1,
}
model.y = pyo.Var(N, within=pyo.Binary, initialize = initialize_dict)
# model.y = pyo.Var(N, within=[0,1])

# print('x["Harlingen", "NYC"]: ', pyo.value(model.x["Harlingen", "NYC"]))
# print('y["Harlingen"]: ', pyo.value(model.y["Harlingen"]))

## Constraints
model.num_warehouses = pyo.Constraint(expr=sum(model.y[n] for n in N) <= P)

def demand_rule(model, m):
    return sum(model.x[n,m] for n in N) == 1
model.demand = pyo.Constraint(M, rule=demand_rule)

def warehouse_active_rule(mdl, n, m):
    return mdl.x[n,m] <= mdl.y[n]
model.warehouse_active = pyo.Constraint(N, M, rule=warehouse_active_rule)

# model.demand.pprint()

# Objective
def obj_rule(model):
    return sum(d[n][m]*model.x[n,m] for n in N for m in M)
model.obj = pyo.Objective(rule=obj_rule)

# model.obj = pyo.Objective(rule=sum(d[n][m]*model.x[n,m] for n in N for m in M))

results = solver.solve(model)
model.x.pprint()

x : Size=12, Index=x_index
    Key                  : Lower : Value : Upper : Fixed : Stale : Domain
      ('Ashland', 'CHI') :     0 :   1.0 :     1 : False : False :  Reals
      ('Ashland', 'HTX') :     0 :   0.0 :     1 : False : False :  Reals
       ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
      ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
    ('Harlingen', 'CHI') :     0 :   0.0 :     1 : False : False :  Reals
    ('Harlingen', 'HTX') :     0 :   1.0 :     1 : False : False :  Reals
     ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
    ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'CHI') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'HTX') :     0 :   0.0 :     1 : False : False :  Reals
       ('Memphis', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'NYC') :     0 :   0.0 :     1 : False : False :  Reals


### Variables Domain
- pyomo predefined virtual sets [here]
- python list
[here]: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html?highlight=UnitInterval#predefined-virtual-sets

### Retrieving optimal values 

In [21]:
# variables
for n in N:
    print(f"{n:10}",pyo.value(model.y[n]))

Harlingen  1.0
Memphis    0.0
Ashland    1.0


In [22]:
# objective
print("objective value: ", pyo.value(model.obj))

objective value:  2745.0


In [20]:
# constraints
print(pyo.value(model.num_warehouses.body))
print(model.num_warehouses.uslack())
print(model.num_warehouses.lslack())

2.0
0.0
inf


### Termination Condition
checking the solver status and
full list of statuses and conditions [here]

[here]: http://www.pyomo.org/blog/2015/1/8/accessing-solver "Solver conditions"

In [21]:
from pyomo.opt import SolverStatus, TerminationCondition

if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print("Termination Condition is Optimal")
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print("Termination Condition is Infeasible")
else:
    # Something else is wrong
    print("Solver Status:" ,  results.solver.status)

Termination Condition is Optimal


## Maximum network flow

warehouse location problem
$$
\begin{align}
max \quad &x_{6,1}\\
s.t.\quad &\sum_{(i,j) \in A} x_{ij} - \sum_{(k,i) \in A} x_{ki} =0 ,\quad \forall i \in N &(1)\\
\quad &  x_{ij} \leq c_{ij}, \quad \forall (i,j) \in A&(2)\\
\quad & x_{ij} \geq 0, \quad \forall (i,j) \in A&\\
\end{align}
$$
<br>
<br>
![Alt](./images/MNF.png?w=1000 "wlp")

In [25]:
## Sets
N = [1,2,3,4,5,6]
A = [(1,2),(1,4),(2,3),(4,5),(3,6),(5,6),(2,5),(4,3),(6,1)]
c ={
    (1,2):8,
    (1,4):9,
    (2,3):7,
    (4,5):2,
    (3,6):6,
    (5,6):5,
    (2,5):4,
    (4,3):3,
    (6,1):100, # don't use inf
}

model = pyo.ConcreteModel()

## Variables
model.x = pyo.Var(N,N,within=pyo.NonNegativeReals)

## Constraints
def capacity_rule(model,i,j):
    if (i,j) in A:
        return model.x[i,j] <= c[(i,j)]
    else:
        return pyo.Constraint.Skip
model.capacity = pyo.Constraint(N, N, rule=capacity_rule)

def equliburium_rule(model,i):
    expr = 0
    for j in N:
        if (i,j) in A:
            expr += model.x[i,j]
    for k in N:
        if (k,i) in A:
            expr -= model.x[k,i]
    return expr == 0
model.equil = pyo.Constraint(N,rule=equliburium_rule)


## common mistake (!) don't use assignmnet for fixiing
## how to fix variables ?!
for i in N:
    for j in N:
        if not (i,j) in A:
            model.x[i,j].fix(0)


## Objective
model.obj = pyo.Objective(expr=model.x[(6,1)], sense=pyo.maximize)
results = solver.solve(model)
pyo.value(model.obj)


11.0

In [27]:
model = pyo.ConcreteModel()

## Variables
model.x = pyo.Var(A,within=pyo.NonNegativeReals)

## Constraints
def capacity_rule(model,i,j):
    return model.x[i,j] <= c[(i,j)]
model.capacity = pyo.Constraint(A, rule=capacity_rule)

def equliburium_rule(model,i):
    expr = 0
    for j in N:
        if (i,j) in A:
            expr += model.x[(i,j)]
    for k in N:
        if (k,i) in A:
            expr -= model.x[(k,i)]
    return expr == 0
model.equil = pyo.Constraint(N,rule=equliburium_rule)

## Objective
model.obj = pyo.Objective(expr=model.x[6,1], sense=pyo.maximize)

## Dual values
## dual values are not captured by default, it should be signaled before optimization
## this variables should be named exactly "dual" and "rc" 
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
model.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)


results = solver.solve(model)
print(pyo.value(model.obj))

# print ("Duals")
# print ("  Capacity Constraint")
# for index in model.capacity:
#     print ("      ", index, model.dual[model.capacity[index]])
# print ("  Equiliburium Constraint")
# for index in model.equil:
#     print ("      ", index, model.dual[model.equil[index]])

print ("Reduced Costs")
print ("  X variables")
for index in model.x:
    print ("      ", index, model.rc[model.x[index]])

# for (i,j) in A:
#     if pyo.value(model.x[(i,j)]) == c[i,j]:
#         print(i,j)



11.0
Reduced Costs
  X variables
       (1, 2) 0.0
       (1, 4) 0.0
       (2, 3) 0.0
       (4, 5) 0.0
       (3, 6) 0.0
       (5, 6) 0.0
       (2, 5) 0.0
       (4, 3) 0.0
       (6, 1) 0.0


## NEOS
- Internet-based client-server application that provides free access to a library of optimization solvers
- University of Wisconsin-Madison
- limitations
    - 3 GB RAM
    - maximum run time of 8 hours per job
    - output limited to 100 MB
    - maximum of 4 threads per job

In [28]:
import os

# provide an email address
os.environ['NEOS_EMAIL'] = 'sina.hkazemi@gmail.com'

# formulate optimization model
model = pyo.ConcreteModel()

## Variables
model.x = pyo.Var(within=pyo.NonNegativeReals)
model.y = pyo.Var(domain=pyo.NonNegativeReals)

## Constraints
model.con1 = pyo.Constraint(expr=model.x + 4*model.y <= 1 )
model.con2 = pyo.Constraint(expr=4*model.x + model.y <= 1)

## Objective
model.obj = pyo.Objective(expr=model.x + model.y, sense=pyo.maximize)

solver_manager = pyo.SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='cplex')
print(pyo.value(model.obj))
print(results)

0.4

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: CPLEX 20.1.0.0\x3a optimal solution; objective 0.4; 2 dual simplex iterations (1 in phase I)
  Termination condition: optimal
  Id: 0
Solution: 
- number of solutions: 0
  number of solutions displayed: 0




## resoucrses:
- Pyomo Documentation [1]
- Pyomo — Optimization Modeling in Python (Third Edition 2021) book [2]

[1]: https://pyomo.readthedocs.io/en/stable/ "Pyomo Documentation"
[2]: https://u1lib.org/book/11924275/1825ce "Pyomo Book"